In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from pprint import pprint
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score, confusion_matrix
import warnings
!pip install kaggler
from kaggler.preprocessing import LabelEncoder
from kaggler.model import AutoLGB

     |████████████████████████████████| 829kB 7.7MB/s 
  Created wheel for kaggler: filename=Kaggler-0.9.6-cp37-cp37m-linux_x86_64.whl size=2959086 sha256=7c17edb59dda8324569b8128855f951ceed339f3833d57b5b36cfc517c30627c
  Stored in directory: /root/.cache/pip/wheels/75/06/a9/f2a7b621e4c6b0571cec751c462683e7528d4fd0bb51060b4f
  Created wheel for ml-metrics: filename=ml_metrics-0.1.4-cp37-none-any.whl size=7850 sha256=86e9d1d891e9f98f60b1bb2898334183f4e6989791d90d36802d6556395b7c3b
  Stored in directory: /root/.cache/pip/wheels/b3/61/2d/776be7b8a4f14c5db48c8e5451451cabc58dc6aa7ee3801163
Successfully built kaggler ml-metrics


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
import kaggler
print(kaggler.__version__)

0.9.6


In [4]:
pd.set_option('max_columns', 100)
warnings.simplefilter('ignore')

### Load Data

In [5]:
algo_name = 'esb'
model_name = f'{algo_name}'

predict_val_file = f'{model_name}.val.txt'
predict_tst_file = f'{model_name}.tst.txt'
submission_file = f'{model_name}.sub.csv'

index_col = 'index'
target_col = 'credit'

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [29]:
trn = pd.read_csv('/content/drive/MyDrive/DACON/credit/train.csv', index_col=index_col)
print(trn.shape)
trn.head()

(26457, 19)


,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month,credit
index,,,,,,,,,,,,,,,,,,,
0,F,N,N,0,202500.0,Commercial associate,Higher education,Married,Municipal apartment,-13899,-4709,1,0,0,0,NaN,2.0,-6.0,1.0
1,F,N,Y,1,247500.0,Commercial associate,Secondary / secondary special,Civil marriage,House / apartment,-11380,-1540,1,0,0,1,Laborers,3.0,-5.0,1.0
2,M,Y,Y,0,450000.0,Working,Higher education,Married,House / apartment,-19087,-4434,1,0,1,0,Managers,2.0,-22.0,2.0
3,F,N,Y,0,202500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-15088,-2092,1,0,1,0,Sales staff,2.0,-37.0,0.0
4,F,Y,Y,0,157500.0,State servant,Higher education,Married,House / apartment,-15037,-2105,1,0,0,0,Managers,2.0,-26.0,2.0


In [30]:
tst = pd.read_csv('/content/drive/MyDrive/DACON/credit/test.csv', index_col=index_col)
print(tst.shape)
tst.head()

(10000, 18)


,gender,car,reality,child_num,income_total,income_type,edu_type,family_type,house_type,DAYS_BIRTH,DAYS_EMPLOYED,FLAG_MOBIL,work_phone,phone,email,occyp_type,family_size,begin_month
index,,,,,,,,,,,,,,,,,,
26457,M,Y,N,0,112500.0,Pensioner,Secondary / secondary special,Civil marriage,House / apartment,-21990,365243,1,0,1,0,NaN,2.0,-60.0
26458,F,N,Y,0,135000.0,State servant,Higher education,Married,House / apartment,-18964,-8671,1,0,1,0,Core staff,2.0,-36.0
26459,F,N,Y,0,69372.0,Working,Secondary / secondary special,Married,House / apartment,-15887,-217,1,1,1,0,Laborers,2.0,-40.0
26460,M,Y,N,0,112500.0,Commercial associate,Secondary / secondary special,Married,House / apartment,-19270,-2531,1,1,0,0,Drivers,2.0,-41.0
26461,F,Y,Y,0,225000.0,State servant,Higher education,Married,House / apartment,-17822,-9385,1,1,0,0,Managers,2.0,-8.0


In [31]:
sub = pd.read_csv('/content/drive/MyDrive/DACON/credit/sample_submission.csv', index_col=index_col)
print(sub.shape)
sub.head()

(10000, 3)


,0,1,2
index,,,
26457,0,0,0
26458,0,0,0
26459,0,0,0
26460,0,0,0
26461,0,0,0


## Feature Engineering

In [32]:
# Feature Selection(shap)
cat_cols = ['reality', 'occyp_type','income_type']
num_cols = ['income_total', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'begin_month']
feature_cols = num_cols + cat_cols
print(len(feature_cols), len(cat_cols), len(num_cols))

7 3 4


### Data 전처리 (occyp_type)

In [33]:
def occyptype_pre(data):
  data.loc[data["DAYS_EMPLOYED"] >= 0,'occyp_type'] = "No job"
  data["occyp_type"] = data["occyp_type"].fillna("unknown")
  return

In [34]:
occyptype_pre(trn)
occyptype_pre(tst)

### 빈도수가 적은 data 처리

In [35]:
# 빈도수가 적은 카테고리 확인(1% 기준)
# staff 직군으로 묶기(빈도수가 낮은 집단이 모두 staff -> sales staff로 묶어서 진행해보자)
def find_minority_cats(cat_cols, data_df, composite_category='Sales staff', threshold=0.03):
    """ Find minority categories for each feature column, and create a 
        dictionary that maps those to selected composite category """
    minority_col_dict = {}
    minority_mapping_dict = {}
    
    # find all feature categories with less than 1% proportion
    for feature in cat_cols:
        minority_col_dict[feature] = []
        minority_mapping_dict[feature] = {}
        
        for category, proportion in data_df[feature].value_counts(normalize=True).iteritems():
            if proportion < threshold:
                minority_col_dict[feature].append(category)
                
                # map those minority cats to chosen composite feature
                minority_mapping_dict[feature] = { x : composite_category for x 
                                                  in minority_col_dict[feature]}
                
    return minority_mapping_dict, minority_col_dict

In [36]:
cat_min_mappings, minority_cols = find_minority_cats(["occyp_type"], trn)
minority_cols

{'occyp_type': ['Cooking staff',
  'Security staff',
  'Cleaning staff',
  'Private service staff',
  'Low-skill Laborers',
  'Waiters/barmen staff',
  'Secretaries',
  'Realty agents',
  'HR staff',
  'IT staff']}

In [37]:
trn["occyp_type"] = trn["occyp_type"].replace(cat_min_mappings["occyp_type"])
tst["occyp_type"] = tst["occyp_type"].replace(cat_min_mappings["occyp_type"])

In [38]:
special_type = ['High skill tech staff','Medicine staff','Accountants']

def specialist(data):
  data.loc[data["occyp_type"] == 'High skill tech staff','occyp_type'] = "specialist"
  data.loc[data["occyp_type"] == 'Medicine staff','occyp_type'] = "specialist"
  data.loc[data["occyp_type"] == 'Accountants','occyp_type'] = "specialist"

  data["occyp_type"] = data["occyp_type"].fillna("unknown")
  return

In [39]:
specialist(trn)
specialist(tst)

In [40]:
trn['occyp_type'].unique()

array(['unknown', 'Laborers', 'Managers', 'Sales staff', 'specialist',
       'Core staff', 'Drivers', 'No job'], dtype=object)

In [41]:
lbe = LabelEncoder(min_obs=10)
trn[cat_cols] = lbe.fit_transform(trn[cat_cols])
tst[cat_cols] = lbe.transform(tst[cat_cols])

In [42]:
scaler = StandardScaler()
trn[num_cols] = scaler.fit_transform(trn[num_cols])
tst[num_cols] = scaler.transform(tst[num_cols])

In [43]:
trn= trn.drop(['child_num', 'FLAG_MOBIL', 'house_type', 'edu_type', 'gender', 'phone', 'email', 'family_type', 'family_size', 'car', 'work_phone'],axis=1)
tst= tst.drop(['child_num', 'FLAG_MOBIL', 'house_type', 'edu_type', 'gender', 'phone', 'email', 'family_type', 'family_size', 'car', 'work_phone'],axis=1)

In [44]:
trn

,reality,income_total,income_type,DAYS_BIRTH,DAYS_EMPLOYED,occyp_type,begin_month,credit
index,,,,,,,,
0,1,0.149136,2,0.490075,-0.463930,3,1.215231,1.0
1,0,0.590848,2,1.089621,-0.440878,1,1.275620,1.0
2,0,2.578550,1,-0.744719,-0.461929,6,0.249003,2.0
3,0,0.149136,2,0.207081,-0.444893,0,-0.656836,0.0
4,0,-0.292575,4,0.219220,-0.444988,6,0.007446,2.0
...,...,...,...,...,...,...,...,...
26452,1,0.369992,4,0.923252,-0.444108,5,1.456788,1.0
26453,0,-0.071719,1,0.158765,-0.447679,3,-1.260729,2.0
26454,1,1.032559,1,1.398558,-0.444333,5,0.067835,2.0


### Base Model

In [45]:
n_est = 1000
seed = 42
n_fold = 5
n_class = 3

lgb_params = {
    'metric': 'multi_logloss',
    'n_estimators': n_est,
    'objective': 'multiclass',
    'random_state': seed,
    'learning_rate': 0.01,
    'min_child_samples': 20,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
    'num_leaves': 63,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'num_class': n_class
}

xgb_params = {
    'metric': 'mlogloss',
    'objective': 'multi:softprob',
    'subsample': 0.8,
    'colsample_bytree': 0.8,
    'learning_rate': 0.01,
    'random_state': seed,
    'num_class': n_class,
    'max_depth': 6,
    'n_estimators': n_est,
    'min_child_samples': 20,
    'reg_alpha': 3e-5,
    'reg_lambda': 9e-2,
}

rf_params = {
    'max_depth': 15,
    'min_samples_leaf': 8,
    'random_state': seed
}

In [46]:
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier


base_models = {'rf': RandomForestClassifier(**rf_params), 
               'lgb': LGBMClassifier(**lgb_params),
               'xgb': XGBClassifier(),
               'et': ExtraTreesClassifier(bootstrap=True, 
                                          criterion='entropy', 
                                          max_features=0.55, 
                                          min_samples_leaf=8, 
                                          min_samples_split=4, 
                                          n_estimators=100)}

In [47]:
from copy import copy

X = trn[feature_cols]
y = trn[target_col]
X_tst = tst[feature_cols]

cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)

p_dict = {}
p_tst_dict = {}
for name in base_models:
    print(f'Training {name}:')
    p = np.zeros((X.shape[0], n_class), dtype=float)
    p_tst = np.zeros((X_tst.shape[0], n_class), dtype=float)
    for i, (i_trn, i_val) in enumerate(cv.split(X, y)):
        clf = copy(base_models[name])
        clf.fit(X.iloc[i_trn], y[i_trn])

        p[i_val] = clf.predict_proba(X.iloc[i_val])
        p_tst += clf.predict_proba(X_tst) / n_fold

    p_dict[name] = p
    p_tst_dict[name] = p_tst
    print(f'\tCV Log Loss: {log_loss(y, p):.6f}')

Training rf:
	CV Log Loss: 0.769272
Training lgb:
	CV Log Loss: 0.754216
Training xgb:
	CV Log Loss: 0.798973
Training et:
	CV Log Loss: 0.799640


### Stacking

In [48]:
X = pd.DataFrame(np.hstack([x for _, x in p_dict.items()]))
X_tst = pd.DataFrame(np.hstack([x for _, x in p_tst_dict.items()]))

p = np.zeros((X.shape[0], n_class), dtype=float)
p_tst = np.zeros((X_tst.shape[0], n_class), dtype=float)
for i_cv, (i_trn, i_val) in enumerate(cv.split(X, y)):
    if i_cv == 0:
        clf = AutoLGB(objective='multiclass', metric='multi_logloss', params={'num_class': n_class}, 
                      feature_selection=False, n_est=10000)
        clf.tune(X.iloc[i_trn], y[i_trn])
        n_best = clf.n_best
        features = clf.features
        params = clf.params
        print(f'best iteration: {n_best}')
        print(f'selected features ({len(features)}): {features}')        
        pprint(params)
        clf.fit(X.iloc[i_trn], y[i_trn])
    else:
        train_data = lgb.Dataset(X[features].iloc[i_trn], label=y[i_trn])
        clf = lgb.train(params, train_data, n_best, verbose_eval=100)
    
    p[i_val] = clf.predict(X[features].iloc[i_val])
    p_tst += clf.predict(X_tst[features]) / n_fold

100%|██████████| 100/100 [01:40<00:00,  1.01s/it, best loss: 0.7344364920054137]
best iteration: 197
selected features (12): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]
{'bagging_fraction': 0.6000000000000001,
 'bagging_freq': 1,
 'boosting': 'gbdt',
 'feature_fraction': 0.6000000000000001,
 'feature_pre_filter': False,
 'lambda_l1': 0,
 'lambda_l2': 0,
 'learning_rate': 0.029613606726953456,
 'max_depth': 4,
 'metric': 'multi_logloss',
 'min_child_samples': 10,
 'num_class': 3,
 'num_leaves': 127,
 'num_threads': -1,
 'objective': 'multiclass',
 'seed': 42,
 'verbosity': -1}


In [49]:
print(f'CV Log Loss: {log_loss(y, p):.6f}')
np.savetxt(predict_val_file, p, fmt='%.6f')
np.savetxt(predict_tst_file, p_tst, fmt='%.6f')

CV Log Loss: 0.744087


### Submit

In [52]:
sub[sub.columns] = p_tst
sub.head()

,0,1,2
index,,,
26457,0.077375,0.133167,0.789458
26458,0.167190,0.158597,0.674213
26459,0.047630,0.079746,0.872625
26460,0.101825,0.110927,0.787248
26461,0.084352,0.238930,0.676718


In [53]:
sub.to_csv(submission_file)

In [54]:
submission_file

'esb.sub.csv'